In this notebook several methods for solving sudokus are implemented.

* In the ***Human rules*** section the implemented rules are the ones I use when solving a sudoku by my own. They are separated in 3 levels of complexity and, using the most advanced ones is possible to solve every sudoku.

* In the ***Backtraking*** section the backtraking algorithm is implemented (https://medium.com/swlh/simple-sudoku-with-backtracking-bb4813ddabb1)

# Libraries and functions

In [1]:
import numpy as np
import copy

In [2]:
def make_squares(sudoku):

  if type(sudoku) != type(np.arange(1)):
    print("The argument is not an array")
    return "The argument is not an array"

  if sudoku.shape != (9,9):
    print("Array doesn't have correct dimensions (9x9)")
    return ("Array doesn't have correct dimensions (9x9")
  
  squares = []
  squares.append(sudoku[0:3,0:3])
  squares.append(sudoku[0:3,3:6])
  squares.append(sudoku[0:3,6:9])
  squares.append(sudoku[3:6,0:3])
  squares.append(sudoku[3:6,3:6])
  squares.append(sudoku[3:6,6:9])
  squares.append(sudoku[6:9,0:3])
  squares.append(sudoku[6:9,3:6])
  squares.append(sudoku[6:9,6:9])

  return squares

def find_duplicates_in_matrix(matrix, count_zeros):
  duplicates = []
  for row in matrix:
    unique_list = []
    row_duplicates = []
    for number in row:
      if number not in unique_list:
        unique_list.append(number)
      else:
        if count_zeros:
          row_duplicates.append(number)
        elif number!=0:
          row_duplicates.append(number)

    duplicates.append(row_duplicates)
    
  return duplicates

def check_sudoku(sudoku, count_zeros=True, verbose=False):
  correct_sudoku = True

  # Check rows
  row_duplicates = find_duplicates_in_matrix(sudoku, count_zeros)
  if sum([len(row) for row in row_duplicates]) != 0:
    print("1 or more elements repeated in a row")
    if verbose:
      print(row_duplicates)
    correct_sudoku = False

  # Check columns
  col_duplicates = find_duplicates_in_matrix(sudoku.transpose(), count_zeros)
  if sum([len(row) for row in col_duplicates]) != 0:
    print("1 or more elements repeated in a column")
    if verbose:
      print(col_duplicates)
    correct_sudoku = False
    
  # Check squares
  squares = make_squares(sudoku)
  square_matrix = np.zeros((9,9), dtype=int)
  for index, square in enumerate(squares):
    square_matrix[index,:] = square.flatten()

  square_duplicates = find_duplicates_in_matrix(square_matrix, count_zeros)
  if sum([len(row) for row in square_duplicates]) != 0:
    print("1 or more elements repeated in a square")
    if verbose:
      print(square_duplicates)
    correct_sudoku = False

  return correct_sudoku

def assing_square_index(row_index, col_index):
  square_index = -1
  if row_index < 3:
    if col_index < 3:
      square_index = 0
    if 3 <= col_index < 6:
      square_index = 1
    if 6 <= col_index < 9:
      square_index = 2
  elif 3 <= row_index < 6:
    if col_index < 3:
      square_index = 3
    if 3 <= col_index < 6:
      square_index = 4
    if 6 <= col_index < 9:
      square_index = 5
  elif 6 <= row_index < 9:
    if col_index < 3:
      square_index = 6
    if 3 <= col_index < 6:
      square_index = 7
    if 6 <= col_index < 9:
      square_index = 8
  return square_index

def try_number(sudoku, number, row_index, col_index):
  correct_number = 1

  correct_number = 0 if number in sudoku[row_index,:] else correct_number
  correct_number = 0 if number in sudoku[:, col_index] else correct_number

  squares = make_squares(sudoku)
  square_index = assing_square_index(row_index, col_index)
  correct_number = 0 if number in squares[square_index] else correct_number

  return correct_number

def square_coords_to_cartesian(square_index, square_position):
  # Minimum row and column
  switcher_1 = {
        0: [0,0],
        1: [0,3],
        2: [0,6],
        3: [3,0],
        4: [3,3],
        5: [3,6],
        6: [6,0],
        7: [6,3],
        8: [6,6],
    } 
  # Adding
  switcher_2 = {
        0: [0,0],
        1: [0,1],
        2: [0,2],
        3: [1,0],
        4: [1,1],
        5: [1,2],
        6: [2,0],
        7: [2,1],
        8: [2,2],
    } 

  cartesian_position = np.asarray(switcher_1.get(square_index, "Invalid square index"))
  cartesian_position = np.add(cartesian_position, np.asarray(switcher_2.get(square_position, "Invalid square position")))
  return cartesian_position

# Sudoku samples

## BEGINNER

In [3]:
sudoku = np.asarray([[3,9,0,6,1,5,0,0,7],
                     [4,1,5,9,7,2,0,3,8],
                     [2,7,6,0,0,4,5,0,1],
                     [7,8,2,0,6,3,0,1,9],
                     [0,5,3,0,4,9,0,0,2],
                     [9,4,1,2,8,7,0,6,5],
                     [0,2,7,3,0,0,0,0,0],
                     [1,0,4,7,5,0,9,2,0],
                     [8,6,9,0,2,1,7,5,3]])

## EASY

In [4]:
sudoku = np.asarray([[6,8,5,0,3,0,2,9,4],
                     [0,0,0,0,9,2,0,0,5],
                     [0,0,3,0,5,6,0,7,0],
                     [2,1,9,6,8,3,5,4,0],
                     [4,5,0,9,2,0,0,8,0],
                     [0,0,8,0,0,5,1,2,0],
                     [0,0,0,0,0,9,0,1,0],
                     [1,9,0,0,7,0,4,6,0],
                     [8,7,6,3,1,0,9,0,0]])

## MEDIUM

In [5]:
sudoku = np.asarray([[0,0,0,7,9,3,0,0,0],
                     [0,9,0,5,2,1,4,0,0],
                     [7,0,0,0,0,0,0,0,9],
                     [2,0,0,4,3,0,7,0,0],
                     [0,0,9,8,1,7,3,5,0],
                     [3,0,0,0,5,0,0,0,8],
                     [0,6,0,0,8,5,0,7,0],
                     [0,0,0,0,6,2,8,9,1],
                     [0,8,0,9,0,0,0,0,0]])

## HARD

In [6]:
sudoku = np.asarray([[0,0,6,0,8,0,1,0,0],
                     [0,0,5,6,0,0,0,0,0],
                     [9,0,0,0,0,1,4,0,3],
                     [0,1,0,2,0,0,3,0,0],
                     [0,0,3,0,0,4,0,8,0],
                     [2,0,0,0,9,0,0,0,0],
                     [0,0,0,0,0,2,0,7,0],
                     [0,6,0,0,0,0,0,3,0],
                     [0,2,0,9,7,0,0,0,0]])

In [7]:
sudoku = np.asarray([[9,0,7,0,8,0,0,0,0],
                     [4,0,0,0,0,0,9,2,0],
                     [3,0,6,0,0,9,8,1,0],
                     [8,0,0,0,0,6,0,0,9],
                     [5,7,0,0,9,0,0,6,8],
                     [6,9,1,8,5,3,7,4,2],
                     [7,0,0,5,0,0,2,9,0],
                     [2,0,0,9,0,0,0,7,0],
                     [1,0,9,2,0,7,6,8,0]])

# Human rules


## Basic

Check cell by cell if only one number can be written or not --> basic checking if same number is in the same row, column or square

In [8]:
n_iter, max_iters = 1, 10
sudoku_to_solve = sudoku
solved_sudoku = copy.deepcopy(sudoku_to_solve) 
print("There are ", sudoku.size - np.count_nonzero(sudoku)," numbers left to be writen")

while (solved_sudoku.size - np.count_nonzero(solved_sudoku)>0 and n_iter<=max_iters):

  for row_index in range(9):
    for col_index in range(9):
      cell = sudoku_to_solve[row_index, col_index]
      if cell==0:
        tried_numbers = [try_number(sudoku_to_solve, number, row_index, col_index) for number in range(1,10)]
        if sum(tried_numbers) == 1:
          correct_number = tried_numbers.index(1) + 1
          solved_sudoku[row_index, col_index] = correct_number
          # print("The number  ",correct_number," has been writen in ", row_index, col_index)
        elif sum(tried_numbers) == 0:
          print("ERROR: The algorithm couldn't find a number that fits in")

  print("Iteration no.", n_iter, "--> There are ", solved_sudoku.size - np.count_nonzero(solved_sudoku)," numbers left to be writen")
  sudoku_to_solve = solved_sudoku
  n_iter+=1

if solved_sudoku.size - np.count_nonzero(solved_sudoku) == 0:
  print("\n*** ¡Sudoku has been solved! ", check_sudoku(solved_sudoku))
  display(solved_sudoku)
else:
  print("Sudoku couldn't be solved, the maximum number of iterations has been reached")

There are  40  numbers left to be writen
Iteration no. 1 --> There are  40  numbers left to be writen
Iteration no. 2 --> There are  40  numbers left to be writen
Iteration no. 3 --> There are  40  numbers left to be writen
Iteration no. 4 --> There are  40  numbers left to be writen
Iteration no. 5 --> There are  40  numbers left to be writen
Iteration no. 6 --> There are  40  numbers left to be writen
Iteration no. 7 --> There are  40  numbers left to be writen
Iteration no. 8 --> There are  40  numbers left to be writen
Iteration no. 9 --> There are  40  numbers left to be writen
Iteration no. 10 --> There are  40  numbers left to be writen
Sudoku couldn't be solved, the maximum number of iterations has been reached


## Medium

In each row/column/square check the possible positions of each number.

In each column of the ***position_matrix*** is represent

In each column of the ***positions_matrix*** is represented one position of each row/column/square of the sudoku. And each row represents each tested number, the 9 numbers are tested in the 9 positions (in practice less, because they are only tested in the free positions) and 0 or 1 is set depending on whether or not each tested number can go in that cell.

After filling the ***positions_matrix*** with 0 and 1, 2 conditions are checked to be able to add numbers to the sudoku:

* If in a **column** of the ***position_matrix*** there is only one box with the value one it will mean that only that number can go in that box, so it will be added to the sudoku.

* If in a **row** of the ***position_marix*** there is only one box with value 1 it means that the corresponding number can only go in that position of the row/columns/square, so it will be added to the sudoku.

In [9]:
n_iter, max_iters = 1, 10
sudoku_to_solve = sudoku
solved_sudoku = copy.deepcopy(sudoku_to_solve) 
print("There are ", sudoku.size - np.count_nonzero(sudoku)," numbers left to be writen")

while (solved_sudoku.size - np.count_nonzero(solved_sudoku)>0 and n_iter<=max_iters):

  # Rows
  for row_index in range(9):
    positions_matrix = np.zeros((9,9), dtype=int)
    for col_index in range(9):
      cell = solved_sudoku[row_index, col_index]
      if cell==0:
        positions_matrix[:,col_index] = [try_number(solved_sudoku, number, row_index, col_index) for number in range(1,10)]
    for index, row in enumerate(positions_matrix):
      if sum(row) == 1:
        position = np.where(row == 1)[0][0]
        solved_sudoku[row_index, position] = index+1
        # print("The number ",index+1," has been writen in the position ", position+1, " of the row no. ", row_index+1)
    for index, col in enumerate(positions_matrix.transpose()):
      if sum(col) == 1:
        position = np.where(col == 1)[0][0]
        solved_sudoku[row_index, index] = position+1

  # Columns
  for col_index in range(9):
    positions_matrix = np.zeros((9,9), dtype=int)
    for row_index in range(9):
      cell = solved_sudoku[row_index, col_index]
      if cell==0:
        positions_matrix[:,row_index] = [try_number(solved_sudoku, number, row_index, col_index) for number in range(1,10)]
    for index, col in enumerate(positions_matrix):
      if sum(col) == 1:
        position = np.where(col == 1)[0][0]
        solved_sudoku[position, col_index] = index+1
        # print("The number ",index+1," has been writen in the position ", position+1, " from the column no. ", col_index+1)

  # Squares
  squares = make_squares(solved_sudoku)
  for square_index in range(9):
    positions_matrix = np.zeros((9,9), dtype=int)
    for square_position in range(9):
      cell = squares[square_index].flatten()[square_position]
      # print(cell)
      if cell==0:
        row_index, col_index = square_coords_to_cartesian(square_index, square_position)   
        positions_matrix[:,square_position] = [try_number(solved_sudoku, number, row_index, col_index) for number in range(1,10)]

    for index, row in enumerate(positions_matrix):
      if sum(row) == 1:
        row_index, col_index = square_coords_to_cartesian(square_index, np.where(row == 1)[0][0])
        solved_sudoku[row_index, col_index] = index+1
        # print("The number ",index+1," has been writen in the position ", position+1, " from the square no.o ", square_index+1)
    
  print("Iteration no. ", n_iter, "--> There are ", solved_sudoku.size - np.count_nonzero(solved_sudoku)," numbers left to be writen")
  # sudoku_to_solve = solved_sudoku
  n_iter+=1

if solved_sudoku.size - np.count_nonzero(solved_sudoku) == 0:
  print("\n*** The sudoku has been solved! ", check_sudoku(solved_sudoku))
  display(solved_sudoku)
else:
  print("Sudoku couldn't be solved, the maximum number of iterations has been reached")

There are  40  numbers left to be writen
Iteration no.  1 --> There are  40  numbers left to be writen
Iteration no.  2 --> There are  40  numbers left to be writen
Iteration no.  3 --> There are  40  numbers left to be writen
Iteration no.  4 --> There are  40  numbers left to be writen
Iteration no.  5 --> There are  40  numbers left to be writen
Iteration no.  6 --> There are  40  numbers left to be writen
Iteration no.  7 --> There are  40  numbers left to be writen
Iteration no.  8 --> There are  40  numbers left to be writen
Iteration no.  9 --> There are  40  numbers left to be writen
Iteration no.  10 --> There are  40  numbers left to be writen
Sudoku couldn't be solved, the maximum number of iterations has been reached


In [10]:
solved_sudoku

array([[9, 0, 7, 0, 8, 0, 0, 0, 0],
       [4, 0, 0, 0, 0, 0, 9, 2, 0],
       [3, 0, 6, 0, 0, 9, 8, 1, 0],
       [8, 0, 0, 0, 0, 6, 0, 0, 9],
       [5, 7, 0, 0, 9, 0, 0, 6, 8],
       [6, 9, 1, 8, 5, 3, 7, 4, 2],
       [7, 0, 0, 5, 0, 0, 2, 9, 0],
       [2, 0, 0, 9, 0, 0, 0, 7, 0],
       [1, 0, 9, 2, 0, 7, 6, 8, 0]])

## Advanced

To be done, it's being developed.

Some rules to be implemented are:

* When ***n*** numbers can fit in ***n*** cells of a row/column/square but the position within those cells is unknown. These cells should count as filled when checking other numbers.
* When a number can be in 2 or 3 positions inside a square, but those positions are aligned (they make a row or a column) this information can be used to deduce the position of that number in other square.


# Backtracking

* One by one, the algorithm will try to find a correct number of each zero cell, one that doesn't appear in that row, columns or aquare.
* If the tried number is correct, the algorithm goes to the next zero cell to try to find a correct number there. 
* If no correct number can be found in a zero cell, the algorith goes to the previous zero cell and try to find another correct number. 
* The algorithm will go back and back in the zero cells until it finds a correct number for each zero cell.

In [11]:
sudoku = np.asarray([[0,0,6,0,8,0,1,0,0],
                     [0,0,5,6,0,0,0,0,0],
                     [9,0,0,0,0,1,4,0,3],
                     [0,1,0,2,0,0,3,0,0],
                     [0,0,3,0,0,4,0,8,0],
                     [2,0,0,0,9,0,0,0,0],
                     [0,0,0,0,0,2,0,7,0],
                     [0,6,0,0,0,0,0,3,0],
                     [0,2,0,9,7,0,0,0,0]])

In [12]:
sudoku_to_solve = copy.deepcopy(sudoku)
n_step, max_steps = 1, 10000
sudoku_finished = False

# Get coordinates of the zeros
it = np.nditer(sudoku_to_solve, flags=['multi_index'])
zero_positions = []
for x in it:
  if x == 0:
    zero_positions.append(it.multi_index)

# tried_numbers saves the numbers tried for each position
tried_numbers = np.zeros((len(zero_positions),9))

position = 0
n_positions = len(zero_positions)

while not sudoku_finished and n_step<max_steps:

  row_index, col_index = zero_positions[position]

  found_number = False
  for number in range(1,10):
    if number in tried_numbers[position]:
      continue
    elif try_number(sudoku_to_solve, number, row_index, col_index):
      tried_numbers[position,np.where(tried_numbers[position]==0)[0][0]] = number
      sudoku_to_solve[row_index, col_index] = number
      # print("The number %d has been writen in the position %d" % (number, position))
      position+=1
      found_number = True
      break
    elif not 0 in tried_numbers[position]:
      print("No se ha podido encontrar solución")
      n_step = max_steps
      break

  if not found_number and position !=0:
    sudoku_to_solve[row_index, col_index] = 0
    tried_numbers[position] = np.zeros(9)
    position-=1
  elif not found_number and position ==0:
    print("A solution couldn't be founded")
    break

  if position == n_positions:
    sudoku_finished = True
    print("\n ***The sudoku has been solved!!", check_sudoku(sudoku_to_solve))
    print("Number of total iterations: ", n_step)
  n_step+=1

display(sudoku_to_solve)


 ***The sudoku has been solved!! True
Number of total iterations:  6635


array([[7, 3, 6, 4, 8, 9, 1, 2, 5],
       [1, 4, 5, 6, 2, 3, 7, 9, 8],
       [9, 8, 2, 7, 5, 1, 4, 6, 3],
       [5, 1, 8, 2, 6, 7, 3, 4, 9],
       [6, 9, 3, 5, 1, 4, 2, 8, 7],
       [2, 7, 4, 3, 9, 8, 5, 1, 6],
       [4, 5, 9, 8, 3, 2, 6, 7, 1],
       [8, 6, 7, 1, 4, 5, 9, 3, 2],
       [3, 2, 1, 9, 7, 6, 8, 5, 4]])

## Backtraking visualization

To be done, it's being developed.